In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
code_dir=Path.cwd()
project_dir=code_dir.parent
input_dir=project_dir/"input"
output_dir=project_dir/"output"
tmp_dir=project_dir/"tmp"
cat_dir = project_dir.parent/"cat12"

In [ ]:
statistics_df = pd.read_csv(input_dir/"data_postcovid.csv")

In [57]:
import seaborn as sns
from scipy.stats import ttest_ind
from pingouin import ancova
import matplotlib.pyplot as plt


def box_plot(data,dv,between,covar,xlabel,ylabel, annotation_line_distance_y=0.1,annotation_line_length=0.1,ylim=None,bonferroni_factor=1):

    sns.set(style="darkgrid", rc={'figure.figsize':(3,4)})

    x=data[between]
    y=data[dv]

    plot = sns.boxplot(x=x, y=y, palette="Blues")
    plot.set_ylabel(ylabel)
    plot.set_xlabel(xlabel)
    plt.xticks([0,1],["Controls", "post SARS-CoV-2"])
    ancova_df = ancova(data=data, dv=dv, covar=covar, between=between)
    pval, cohen_f = float(ancova_df[ancova_df["Source"]==between]["p-unc"]), float(ancova_df[ancova_df["Source"]==between]["F"])
    print("before pval",pval)
    pval = pval * bonferroni_factor
    print("after pval", pval)
    if pval > 1: pval = 1
    elif pval < 0.005: pval = "<0.005"
    else: pval = f"{pval:.3f}"
    
    x1,x2 = 0,1
    y,h,col = data[dv].max() + annotation_line_distance_y,annotation_line_length,"k"

    plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
    plt.text((x1+x2)*.5, y+h+h*0.2, f"$f$ : {cohen_f:.2f}\n$p_{{bonferroni}}$ : {pval}", ha='center', va='bottom', color=col)

    minimum = data[dv].min() -10 
    if minimum < 0: minimum = -5
    maximum = data[dv].max() 
    if maximum > 20 : maximum += 20
    if ylim: 
        minimum = ylim[0]
        maximum = ylim[1]
        plot.set_ylim(minimum, maximum)
    else: plot.set_ylim(minimum , maximum)


    filename="boxplot_" + ylabel.replace(" ","_").lower()

    return plot, ancova_df, filename

# Descriptive stats

In [111]:
import pingouin as pg

def stats_2groups(df,columns,stats,group_col,covar=None):



# df: dataframe with statistics

# columns: columns to describe

# stats: statistical tests to apply to respective columns (length has to match)

# group_col: columns that differentiates groups

# covar: covariates for ancova



    from scipy.stats import ttest_ind,chi2_contingency,mannwhitneyu

    from statsmodels.stats.multitest import multipletests

    import pingouin as pg

    from pingouin import ttest,mwu

    values_group = df[group_col].unique()

    values_group = [int(x) for x in values_group if not pd.isnull(x)]



    data_controls=df[df[group_col]==0]

    data_patients=df[df[group_col]==1]





    df_control_describe = data_controls.describe()

    df_pat_describe = data_patients.describe()

    target_df=pd.DataFrame(columns=columns)

    for idx,col in enumerate(columns):


        stat = stats[idx]



        if df[col].dtypes == "object":


            values = df[col].unique()

            values = [x for x in values if not pd.isnull(x)]

            n_contr_1 = data_controls[data_controls[col] == values[0]].shape[0]

            n_contr_2 = data_controls[data_controls[col] == values[1]].shape[0]

            target_df.loc[f"contr_percent",col]= n_contr_1 / (n_contr_1 + n_contr_2)

            n_pat_1 = data_patients[data_patients[col] == values[0]].shape[0]

            n_pat_2 = data_patients[data_patients[col] == values[1]].shape[0]

            target_df.loc[f"pat_percent",col]= n_pat_1 / (n_pat_1 + n_pat_2)

            chi_stat, p_val, _, _ = chi2_contingency(pd.crosstab(df[group_col],df[col]).T)

            target_df.loc[f"stat",col] = chi_stat

            target_df.loc[f"pval",col] = p_val

        else:


            target_df.loc["contr_mean",col]=df_control_describe[col]["mean"]

            target_df.loc["contr_std",col]=df_control_describe[col]["std"]

            target_df.loc["contr_count",col]=df_control_describe[col]["count"]

            target_df.loc["contr_median",col]=df_control_describe[col]["50%"]

            target_df.loc["contr_IQR25",col]=df_control_describe[col]["25%"]

            target_df.loc["contr_IQR75",col]=df_control_describe[col]["75%"]
            
            target_df.loc["",col]="---"
            
            target_df.loc["pat_mean",col]=df_pat_describe[col]["mean"]

            target_df.loc["pat_std",col]=df_pat_describe[col]["std"]

            target_df.loc["pat_count",col]=df_pat_describe[col]["count"]

            target_df.loc["pat_median",col]=df_pat_describe[col]["50%"]

            target_df.loc["pat_IQR25",col]=df_pat_describe[col]["25%"]

            target_df.loc["pat_IQR75",col]=df_pat_describe[col]["75%"]


        if stat == "ttest": 

            stat_df = ttest(data_controls[col], data_patients[col])
            p_val, statistic = float(stat_df.loc["T-test","p-val"]), float(stat_df.loc["T-test","T"])

        if stat == "mwu": 

            stat_df = mwu(x=data_controls[col],y=data_patients[col])
            p_val, statistic = float(stat_df.loc["MWU","p-val"]), float(stat_df.loc["MWU","RBC"])

        if stat == "ancova": 
         
            stat_df = pg.ancova(df, dv=col, between=group_col, covar=covar)
            p_val, statistic = float(stat_df.loc[0,"p-unc"]), float(stat_df.loc[0,"F"])
            target_df.loc["covariates",col]=covar
            target_df.loc["main_effect",col]=group_col

        target_df.loc[" ",col]="---"

        target_df.loc["stat",col]=statistic

        target_df.loc["pval",col]=p_val

        target_df.loc['p_bonferroni',col] = target_df.loc['pval',col] * len(columns)

        target_df.loc["p_fdr"] = multipletests(target_df.loc["pval"], alpha=0.05, method="fdr_bh")[1]

        target_df.loc['p_bonferroni'][target_df.loc['p_bonferroni']>1] = 1



    print("n (total) controls: ", data_controls.shape[0])

    print("n (total) patients: ", data_patients.shape[0])



    return target_df


In [ ]:
imaging_stats = stats_2groups(statistics_df,["tbss_skeleton_mean_FA", "tbss_skeleton_mean_FAt", "tbss_skeleton_mean_MD",
"tbss_skeleton_mean_FW", "tbss_skeleton_mean_fd", "tbss_skeleton_mean_logfc", "tbss_skeleton_mean_fdc",
"tbss_skeleton_mean_complexity", "thickness_mean", "psmd_global", 'wmh_load_perc'], ['ancova'] * 11, 'cohort_obj', ['age', 'sex', 'years_of_education'])
imaging_stats.to_csv(output_dir.joinpath('postcovid_global_imaging_statistics.csv'))



In [18]:
import seaborn as sns 
import matplotlib.pyplot as plt

def regression_plot(x,y,xlabel,ylabel,xy,ylim=None, bonferroni_factor=None):

    sns.set(style="darkgrid", rc={'figure.figsize':(6,4)})
    
    plot = sns.regplot(x=x, y=y)
    plot.set_ylabel(ylabel)
    plot.set_xlabel(xlabel)

    if ylim: 
        minimum = ylim[0]
        maximum = ylim[1]
        plot.set_ylim(minimum, maximum)

    from scipy.stats import spearmanr
    rsp,pval = spearmanr(x, y, nan_policy="omit")
    if bonferroni_factor: pval = pval*bonferroni_factor
    if pval < 0.005: pval = "<0.005"
    else: pval = f"{pval:.2f}"
    
    plt.annotate(f"$r_{{sp}}$:{rsp:.3f}\n$p_{{bonferroni}}$ : {pval}",xy=xy)

    filename="regression_" + ylabel.replace(" ","_").lower()

    return plot, filename
